In [ ]:
from rutas import data_clean
maestro = data_clean()

<h2 style="text-align:center;">Regresion lineal multivariada</h2>

In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dict_criterios = {"HOMBRE":
    {"BUZO":["COLOR", "ESTIICA", "TIPO"], #TIPO PULLOVER - HOODIE
    "CAMISA": ["ESTETICA", "FIT", "COLOR"],
    "CAMISETA":["COLOR", "ESTETICA", "FIT", "TIPO"], #TIPO MANGA CORGA - MANGA LARGA
    "CHAQUETA": ["COLOR", "ESTETICA"],
    "JEAN": ["COLOR", "ESTETICA"],
    "JOGGER": ["COLOR", "ESTETICA"],
    "SHORTS": ["COLOR", "ESTETICA"],
    "SHORTS PLANO": ["COLOR", "ESTETICA"],
    "PANTALON": ["COLOR"],
    "POLO": ["COLOR", "TEJIDO", "PUNTO CORAZON"],
    "TANK": ["COLOR", "ESTETICA", "TIPO"] #TIPO CHOMPA
    },
    "MUJER":{"BUZO":["COLOR", "ESTIICA", "TIPO", "LARGO", "BASE TELA", "SILUETA"], #TIPO PULLOVER - HOODIE ## VALIDAR SI TENEMOS SOLUETA
    "CAMISA": ["ESTETICA", "TIPO", "COLOR", "LARGO"], #TIPO MANGA CORGA - MANGA LARGA
    "CAMISETA":["COLOR", "ESTETICA", "LARGO", "SILUETA"], #TIPO MANGA CORGA - MANGA LARGA
    "CHAQUETA": ["COLOR", "ESTETICA", "TIPO", "LARGO", "BASE TELA", "SILUETA"],
    "FALDA": ["ESTETICA", "BASE TELA", "LARGO"],
    "HAREM" : ["SILUETA", "MOLDE", "COLOR", "ESTETICA", "BASE TELA"],
    "JEAN": ["COLOR", "ESTETICA", "TIPO", "TIRO"], #TIRO EN LA SUBCATEGORIA
    "JOGGER": ["COLOR", "ESTETICA", "SILUETA", "BASE TELA", "TIPO"],
    "PANTALON": ["COLOR", "ESTETICA", "TIRO", "TIPO"], #TIRO EN LA SUBCATEGORIA
    "R DEPORTIVA": ["COLOR", "ESTETICA", "BASE TELA", "SILUETA"],
    "SHORTS": ["COLOR", "ESTETICA", "SILUETA", "BASE TELA"],
    "SHORTS DENIM": ["COLOR", "ESTETICA", "BASE TELA", "TIRO"],
    "TOP": ["COLOR", "LARGO", "SILUETA", "ESTETICA", "TIPO"],
    "TOP ML": ["COLOR", "LARGO", "SILUETA", "ESTETICA", "TIPO"],
    "VESTIDO": ["BASE TELA", "SILUETA", "LARGO", "ESTETICA", "COLOR", "TIPO"], 
    "VESTIDO DE BAÑO": ["COLOR", "ESTETICA", "MOLDE"]
    }
}

In [ ]:
y = np.array(maestro["calif"])
X = maestro.copy()
X.drop(["calificacion", "calif"], axis=1, inplace=True)

X = X[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]

<h4 style="text-align:center;">Modelo</h4>

In [41]:
(X_train, X_test, y_train, y_test) = train_test_split(
    X,
    y,
    train_size = 0.7)
    
for i in range(1000):
    try:
        (X_train, X_test, y_train, y_test) = train_test_split(
            X,
            y,
            train_size = 0.7)
        pipeline = Pipeline(
                steps=[
                    (
                        "column_transformer",
                        make_column_transformer(
                            (
                                OneHotEncoder(),
                                make_column_selector(dtype_include=object)
                            ), 
                            remainder = "passthrough"
                        ),
                    ),
                    (
                        "selectKBest",
                        SelectKBest(score_func=f_regression),
                    ),
                    (
                        "linearRegression",
                        LinearRegression(),
                    ),
                ],
            )
        param_grid = {
                "selectKBest__k": range(1, len(X.columns)),
            }
        gridSearchCV = GridSearchCV(
                estimator=pipeline,
                param_grid=param_grid,
                scoring="neg_mean_squared_error",
                refit=True,
                return_train_score=False,
            )
        gridSearchCV.fit(X_train, y_train)
        y_train_pred = gridSearchCV.predict(X_train)
        y_test_pred = gridSearchCV.predict(X_test)
        mse_train = mean_squared_error(
            y_train_pred,
            y_train,
        ).round(2)
        mse_test = mean_squared_error(
            y_test_pred,
            y_test,
        ).round(2)

        r_2 = r2_score(y_pred = y_test_pred, y_true = y_test)

        print(f"mse train: {mse_train} \nmse tres: {mse_test} \nEl R2 es {r_2}")
        break
    except: continue 
print(i)

mse train: 1.5 
mse tres: 1.73 
El R2 es -0.07744033056851274
2


<h2 style="text-align:center;">Red Neuronal TensorFlow</h2>

In [92]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Softmax
from keras.metrics import CategoricalAccuracy

In [158]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

y = np.array(maestro["calif"])
X = maestro.copy()
X.drop(["calificacion", "calif"], axis=1, inplace=True)


X = X[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]
X = X.to_numpy()

training_data = make_column_transformer(
                            (
                                OneHotEncoder(),
                                make_column_selector(dtype_include=object)
                            ), 
                            remainder = "passthrough"
                        )
training_data = np.array(training_data)
target_data = y

# Seleccionar las columnas del dataframe que son categóricas
categorical_columns = maestro.select_dtypes(include='object').columns

# Crear una copia del dataframe para evitar modificar el original
df_copy = maestro.copy()

# Crear una instancia de LabelEncoder para cada columna categórica
label_encoders = {col: LabelEncoder() for col in categorical_columns}

# Aplicar el LabelEncoder a cada columna categórica del dataframe
for col in categorical_columns:
    df_copy[col] = label_encoders[col].fit_transform(df_copy[col])

# Seleccionar las columnas del dataframe que se utilizarán como características
features = df_copy[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]

# Seleccionar la columna del dataframe que se utilizará como objetivo
target = df_copy['calif']

# Crear un modelo de redes neuronales con Keras
model = Sequential()

# Añadir una capa de entrada con la misma cantidad de neuronas que de características
model.add(Dense(features.shape[1], input_dim=features.shape[1], activation='relu'))

# Añadir una capa oculta con 16 neuronas
model.add(Dense(16, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(16, activation='relu'))



# Añadir una capa de salida con una sola neurona

model.add(Dense(5, activation='Softmax'))

In [159]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=CategoricalAccuracy())
history = model.fit(features, target, epochs=1000, verbose=0)

In [7]:
f = plt.figure()
f.set_figwidth(30)
f.set_figheight(9)

plt.plot(history.history['categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('categorical_accuracy')
plt.xlabel('epoch')


NameError: name 'history' is not defined

<Figure size 2160x648 with 0 Axes>

<h2 style="text-align:center;">Red Neuronal Sklearn</h2>

In [1]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")


In [3]:
categorical_columns = maestro.select_dtypes(include='object').columns

# Crear una copia del dataframe para evitar modificar el original
df_copy = maestro.copy()

# Crear una instancia de LabelEncoder para cada columna categórica
label_encoders = {col: LabelEncoder() for col in categorical_columns}

# Aplicar el LabelEncoder a cada columna categórica del dataframe
for col in categorical_columns:
    df_copy[col] = label_encoders[col].fit_transform(df_copy[col])

# Seleccionar las columnas del dataframe que se utilizarán como características
features = df_copy[["COLOR", "ESTETICA", "LARGO", "SILUETA"]]

# Seleccionar la columna del dataframe que se utilizará como objetivo
target = df_copy['calif']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=0)

In [9]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu", early_stopping=True,
                    learning_rate="adaptive", hidden_layer_sizes=(3000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.32142857142857145 
El score en el test fue:  0.2755102040816326


In [10]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(3000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.6275510204081632 
El score en el test fue:  0.25510204081632654


In [11]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(30000, 200), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))

El score en el training fue:  0.25510204081632654 
El score en el test fue:  0.14285714285714285


In [12]:
clf = MLPClassifier(solver='adam', alpha=1e-5, activation="relu",
                    hidden_layer_sizes=(50000, 2000), random_state=1)
clf.fit(X_train, y_train)

print("El score en el training fue: ", clf.score(X_train, y_train), "\nEl score en el test fue: ", clf.score(X_test, y_test))